# Import Modules

In [1]:
import pandas as pd
import numpy as np
from pygam import LinearGAM, LogisticGAM, s, f, te
import matplotlib.pyplot as plt
import random
import math
import requests
import tarfile

# Obtain CCDS data
The version of CCDS for GRCh37 is Homo Sapiens 37.1 (code Hs 37.1). We need 2 files: </br>
1. CCDS.current.txt contains information of gene name, gene id (entrez_gene_id), and CCDS gene ID. 
2. CCDS_protein.current.faa contains information of CCDS gene id and amino acid sequence (from which we use 3*len(amino acid) as gene_length) 

The url for obtaining these files from CCDS database: https://ftp.ncbi.nih.gov/pub/CCDS/archive/Hs37.1/


In [2]:
# read in the CCDS current file
CCDS_current = pd.read_csv("CCDS.current_p38.txt", sep="\t")
CCDS_current = CCDS_current[['gene','gene_id', 'ccds_id']]

# process the CCDS protein data
ccds_length = dict()
with open("CCDS_protein.current_p38.faa", "r") as a_file:
    for line in a_file:
        if line[0] == '>':
            index = line.find('|')
            ccds_id = line[1:index]
            ccds_length[ccds_id] = 0
        else:
            aa_partial_length = len(line) - 1
            ccds_length[ccds_id] = ccds_length[ccds_id] + 3*aa_partial_length

data_items = ccds_length.items()
data_list = list(data_items)
CCDS_cDNA_length = pd.DataFrame(data_list)

# joint two table based in ccds_id
CCDS_data = pd.merge(CCDS_current, CCDS_cDNA_length, how="left", left_on=['ccds_id'], right_on = [0]).drop(columns=[0]).rename(columns={1:"cDNA length"})
CCDS_data.head()

,gene,gene_id,ccds_id,cDNA length
0,LINC00115,79854,CCDS1.1,NaN
1,SAMD11,148398,CCDS2.2,2043.0
2,NOC2L,26155,CCDS3.1,2247.0
3,PLEKHN1,84069,CCDS4.1,1833.0
4,HES4,57801,CCDS5.1,663.0


# Read in TCGA dataset

In [4]:
TCGA = pd.read_csv("data_mutations_extended_TCGA.txt", sep="\t")
TCGA = pd.merge(TCGA, CCDS_data, how="left", left_on=['Entrez_Gene_Id'], right_on = ['gene_id'])
sample_list = set(TCGA['Tumor_Sample_Barcode'].tolist())

# Perform filtering for each sample


In [7]:
def sample_filter(sample_id):
    # generate the mutation status for every CCDS gene for each sample
    mutation_data = TCGA.loc[TCGA['Tumor_Sample_Barcode'] == sample_id].reset_index()
    mutation_list = mutation_data['Entrez_Gene_Id'].tolist()
    all_ccds_gene = CCDS_data.copy()
    all_ccds_gene['mutation status'] = 0
    for i in range(len(all_ccds_gene)):
        gene_id = all_ccds_gene.loc[i]['gene_id']
        if gene_id in mutation_list:
            all_ccds_gene.at[i, 'mutation status'] = 1

    # prepare the dataset
    all_ccds_gene = all_ccds_gene.dropna() #drop genes without reported cDNA length
    all_ccds_gene = all_ccds_gene.sort_values(by='cDNA length', ascending=False)
    all_ccds_gene = all_ccds_gene.drop_duplicates(subset='gene', keep="first")

    gene_X = all_ccds_gene['cDNA length']
    gene_Y = all_ccds_gene['mutation status']

    # Fit using logistic GAM. F is monotonic increase cubic spline with 6 knots
    lgam = LogisticGAM(s(0, n_splines = 7, spline_order = 4, constraints = 'monotonic_inc')).fit(gene_X, gene_Y)

    # Find the PMV with reverse logit function
    fitted_value = lgam.partial_dependence(term=0, X=gene_X)
    pmv = [math.exp(x)/(1+math.exp(x)) for x in fitted_value]
    all_ccds_gene['PMV'] = pmv

    # resample
    all_ccds_gene['Number of sample contain'] = [0] *len(all_ccds_gene) #store the number of sample containing that gene as mutatated gene
    gene_list = all_ccds_gene['gene'].tolist()

    for i in range(1000):
        sample_gene = all_ccds_gene.sample(n=max([50, len(mutation_list)]), replace=False, weights=pmv)
        selected_genes = sample_gene['gene'].tolist()
        for gene in selected_genes:
            all_ccds_gene.loc[all_ccds_gene['gene'] == gene, 'Number of sample contain'] += 1

    # Filtering out mutgenes (status = 1) that have >= 50 count 
    final_mut_gene = all_ccds_gene.loc[all_ccds_gene['mutation status']==1]
    return all_ccds_gene

In [ ]:
import time
start_time = time.time()
passenger_gene = list()
for sample_id in sample_list:
    all_ccds_gene = sample_filter(sample_id)
    final_mut_gene = all_ccds_gene.loc[all_ccds_gene['mutation status'] == 1].loc[all_ccds_gene['Number of sample contain'] >= 50]
    filtered_gene = final_mut_gene['gene'].tolist()
    passenger_gene = passenger_gene + filtered_gene
print("--- %s seconds ---" % (time.time() - start_time))

did not converge


C:\ProgramData\Anaconda3\lib\site-packages\pygam\links.py:149: RuntimeWarning: divide by zero encountered in true_divide
  return dist.levels/(mu*(dist.levels - mu))
C:\ProgramData\Anaconda3\lib\site-packages\pygam\pygam.py:591: RuntimeWarning: invalid value encountered in multiply
  return sp.sparse.diags((self.link.gradient(mu, self.distribution)**2 *
